In [1]:
import sys

sys.path.append("..")

%load_ext autoreload
%autoreload complete

In [2]:
from math import ceil
from functools import partial
from pathlib import Path
import json

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import regionmask
from pylipd.lipd import LiPD
import cf_xarray as cfxr

from lmrecon.io import open_mfdataset, save_mfdataset
from lmrecon.plotting import plot_field
from lmrecon.units import convert_thetaot_to_ohc
from lmrecon.grid import GLOBAL_GRID
from lmrecon.util import stack_state, get_data_path, get_base_path
from lmrecon.stats import area_weighted_mean

In [3]:
frequency = "seasonal"
# frequency = "annual"
ds_path = Path() / f"cmip6_{frequency}_anomalies" / "MRI-ESM2-0" / "past1000"
ds = open_mfdataset(get_data_path() / ds_path)["tas"]

In [9]:
pages2k = LiPD()
pages2k.load_from_dir(get_base_path() / "proxies/Pages2kTemperature2_1_2")

In [21]:
pages2k2 = LiPD()
pages2k2.load_from_dir(get_base_path() / "proxies/LiPD_Files")

In [43]:
locs = (
    pages2k2.get_all_locations()
    .sample(frac=1, random_state=np.random.default_rng(46234))
    .rename(columns={"geo_meanLat": "lat", "geo_meanLon": "lon"})
    .reset_index(drop=True)
)
locs["lon"] = locs["lon"] % 360
locs.iloc[:30]

In [44]:
obs = xr.combine_by_coords(
    [
        ds.sel(lat=[locs.lat.iloc[i]], lon=[locs.lon.iloc[i]], method="nearest").stack(
            dict(location=["lat", "lon"])
        )
        for i in range(len(locs.index))
    ]
)
obs

In [45]:
sigma_obs = np.sqrt(0.4)

In [46]:
obs["tas"] += np.random.default_rng(52565641).normal(0, sigma_obs, obs["tas"].shape)

In [10]:
# directory = get_data_path() / f"pseudoobs_{frequency}" / "2024-06-03T01-40-42"
# save_mfdataset(
#     cfxr.encode_multi_index_as_compress(obs, "location"), directory, add_timestamp=False
# )

# json.dump(
#     {
#         "true_dataset": str(ds_path),
#         "sigma_obs": sigma_obs,
#     },
#     (directory / "metadata.json").open("w"),
#     indent=4,
# )

In [47]:
def plot_comparison():
    fig, ax = plt.subplots()

    ds_obs = obs["tas"].sel(time=slice(1250, 1265)).mean("location")
    ax.plot(ds_obs.time, ds_obs)
    
    ds_true = area_weighted_mean(ds.sel(time=slice(1250, 1265)))
    ax.plot(ds_true.time, ds_true)

plot_comparison()

In [41]:
def plot_obs_loc():
    fig, ax = plt.subplots(
        figsize=(8, 4),
        subplot_kw=dict(projection=ccrs.EqualEarth(central_longitude=180)),
    )

    ax.coastlines()
    ax.scatter(obs.lon, obs.lat, transform=ccrs.PlateCarree(), color="C2", s=20)

plot_obs_loc()

# Determining noise level

Determined from annual mean data since seasonal cycle would have to be eliminated from seasonal data.
Annual mean TAS has mean std of 0.66 K (very similar for median std). Commonly used SNR (ratio of stds) is 0.5 (e.g., Smerdon 2012 or Steiger 2014). Noise std is then 1.33 K.

In [ ]:
plt.plot(obs.time, obs["tas"].sel(location=0))

In [ ]:
obs.std("time").mean().compute()

In [ ]:
0.6629 / 0.5